In [7]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict, Annotated, Literal
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field
from dotenv import load_dotenv
import operator
from langchain_core.messages import SystemMessage, HumanMessage



In [4]:
load_dotenv()

True

In [5]:
generator_llm = ChatOpenAI(model = 'gpt-4o-mini')
evaluator_llm = ChatOpenAI(model = 'gpt-4o')
optimizer_llm = ChatOpenAI(model = 'gpt-4o-mini')


In [8]:
class Poststate(TypedDict):
    title: str
    post: str
    evaluation : Literal['approved', 'needs_improvement']
    feedback: str
    iteration: int
    max_iteration: int
    post_history: Annotated[list[str], operator.add]
    feedback_history: Annotated[list[str], operator.add]
    



In [9]:
class Evaluation_schema(BaseModel):
    evaluation: Literal['approved', 'needs_improvement'] = Field(description= 'whether the created post has been approved or needs improvemnt')
    feedback: str = Field(description= 'the detailed feedback of the tweet')

In [10]:
structured_evaluator_llm = evaluator_llm.with_structured_output(Evaluation_schema)

In [15]:
def generate_post(state: Poststate):
    message = [
         SystemMessage(content="You are a funny and clever linkedin influencer."),
        HumanMessage(content=f"""
Write a short, original, and hilarious post on the topic: "{state['title']}".

Rules:
- Do NOT use question-answer format.
- Max 280 characters.
- Use observational humor, irony, sarcasm, or cultural references.
- Think in meme logic, punchlines, or relatable takes.
- Use simple, day to day english
""")
    ]
    post = generator_llm.invoke(message).content
    return {'post': post, 'post_history': [post]}

In [20]:
def evaluate_post(state: Poststate):
    message = [
        SystemMessage(content="You are a ruthless, no-laugh-given linkedin critic. You evaluate linkedin post based on humor, originality, virality, and post format."),
    HumanMessage(content=f"""
Evaluate the following post:

Post: "{state['post']}"

Use the criteria below to evaluate the tweet:

1. Originality – Is this fresh, or have you seen it a hundred times before?  
2. Humor – Did it genuinely make you smile, laugh, or chuckle?  
3. Punchiness – Is it short, sharp, and scroll-stopping?  
4. Virality Potential – Would people retweet or share it?  
5. Format – Is it a well-formed tweet (not a setup-punchline joke, not a Q&A joke, and under 280 characters)?

Auto-reject if:
- It's written in question-answer format (e.g., "Why did..." or "What happens when...")
- It exceeds 280 characters
- It reads like a traditional setup-punchline joke
- Dont end with generic, throwaway, or deflating lines that weaken the humor (e.g., “Masterpieces of the auntie-uncle universe” or vague summaries)

### Respond ONLY in structured format:
- evaluation: "approved" or "needs_improvement"  
- feedback: One paragraph explaining the strengths and weaknesses 
""")
    ]

    output = structured_evaluator_llm.invoke(message)
    return {'evaluation': output.evaluation, 'feedback': output.feedback, 'feedback_history': [output.feedback]}

In [26]:
def optimize_post(state:Poststate):
    message = [
        SystemMessage(content="You punch up linkdin post for virality and humor based on given feedback."),
        HumanMessage(content=f"""
Improve the post based on this feedback:
"{state['feedback']}"

Topic: "{state['title']}"
Original Post:
{state['post']}

Re-write it as a short, viral-worthy linkedin post. Avoid Q&A style and stay under 280 characters.
""")
    ]
    response = optimizer_llm.invoke(message).content
    iteration = state['iteration']+1
    return {'post': response, 'iteration': iteration, 'post_history': [response]}

In [27]:
def evaluate_route(state: Poststate):
    if state['evaluation'] == 'approved' or state['iteration'] >= state['max_iteration']:
        return 'approved'
    else:
        return 'needs_improvement'

In [28]:
graph = StateGraph(Poststate)
graph.add_node("generate_post", generate_post)
graph.add_node('evaluate_post', evaluate_post)
graph.add_node('optimize_post', optimize_post)
graph.add_edge(START, 'generate_post')
graph.add_edge('generate_post', 'evaluate_post')
graph.add_conditional_edges('evaluate_post', evaluate_route, {'approved': END, 'needs_improvement': 'optimize_post'})
graph.add_edge('optimize_post', 'evaluate_post')

workflow = graph.compile()


In [29]:
workflow

In [30]:
initial_state = {
    "title": "Agentic RAG",
    "iteration": 1,
    "max_iteration": 5
}

final_state = workflow.invoke(initial_state)

In [31]:
final_state['post']

'Just unlocked the secret of "Agentic RAG"! It\'s like my cat, Bella, who boldly ignores my pleas while confidently knocking over my coffee. 🐾💥 In leadership, embrace your power—even if it means a little chaos. Success is a mix of confidence and comedy! #LeadershipGoals #CatWisdom'

In [32]:
final_state['iteration']

3